In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
%matplotlib inline
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from lbs.dl.encoders import enc_seq_onehot, enc_label
import pickle
from sklearn.utils import shuffle

In [2]:
df = pickle.load(open('./../3_Train_Test_Split/out/train_data.p', 'rb'))

In [1]:
def encode(df):
    sequences = []
    labels = []
    entries = []
    for entry, seq, label in zip(df.index, df['sequence'].tolist(), df['socket_assignment'].tolist()):
        if len(seq) <= 500:
            pad_left = random.randint(0, 500 - len(seq))
            sequences.append(enc_seq_onehot(seq, pad_length=500, pad_left=pad_left))
            labels.append(enc_label(label, pad_length=500, pad_left=pad_left))
            entries.append(entry)
    return (np.asarray(sequences), np.asarray(labels), entries)

In [2]:
def decode(pred, enc_seq):
    decoded_preds = []
    for entry_pred, entry in zip(pred, enc_seq):
        decoded_pred = []
        for pos_pred, pos_seq in zip(entry_pred, entry):
            if not np.array_equal(pos_seq, np.zeros(20)):
                decoded_pred.append(pos_pred[1])
        decoded_preds.append(np.asarray(decoded_pred))
    return np.asarray(decoded_preds)

In [5]:
from lbs.dl.logger import Logger
from lbs.dl.metrics import total_accuracy
from keras.models import Model
from keras.models import Sequential
from keras.layers import SpatialDropout1D, SpatialDropout2D, MaxPooling1D, MaxPooling2D, Dense, TimeDistributed, Convolution1D, BatchNormalization,Input,merge,LSTM, Dropout,Embedding,Bidirectional
from keras.optimizers import Adam
from keras.regularizers import l2
import keras.backend as K
import numpy as np
def wcc(weights):
    weights = K.variable(weights)
    def loss(y_true, y_pred):
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    return loss
weights = np.array([1,5])

Using TensorFlow backend.


In [6]:
def Model():
    lr = 0.0005
    model = Sequential()
    model.add(Convolution1D(64, 28, padding='same', activation='relu', kernel_regularizer=l2(0.0001), input_shape=(500, 20)))
    model.add(Dropout(0.5))
    model.add(Convolution1D(64, 21, padding='same',  activation='relu', kernel_regularizer=l2(0.0001)))
    model.add(Dropout(0.25))
    model.add(Dense(128))
    model.add(Dense(2, activation='softmax', name='out'))
    adam = Adam(lr=lr)
    model.compile(optimizer=adam, loss=wcc(weights), metrics=[total_accuracy])
    return model

In [7]:
cv_results = {}
for i in range(1, 6):
    print('CV #%s:' % i)
    ch1 = Logger(lab_pos=1, out_path='models/final_seq/', out_fn='final_seq_%s.h5' % (i), out_log='final_seq.txt' )
    callbacks_list = [ch1]
    model = Model()
    ### Get data
    val_df = df[df['val'] == i]
    train_df = df[df['val'] != i]
    train_data= encode(train_df)
    valid_data = encode(val_df)
    ### Train
    h = model.fit(train_data[0], train_data[1],
                  validation_data = (valid_data[0], valid_data[1]),
                  batch_size=64,
                  epochs=50,
                  verbose=0, callbacks=callbacks_list)
    ### Predict
    model.load_weights('models/final_seq/final_seq_%s.h5' % i)
    preds = model.predict(valid_data[0])
    ### Decode predictions and add results to dict
    decoded_preds = decode(preds, valid_data[0])
    for entry, decoded_pred in zip(valid_data[2], decoded_preds):
        assignment  = ''.join(str(int(label)) for label in np.rint(decoded_pred))
        cv_results[entry] = assignment
    print(len(cv_results))

CV #1:
Best F1 score: 0.294 (prec: 0.256, sens: 0.343)
Best F1 score: 0.305 (prec: 0.353, sens: 0.268)
Best F1 score: 0.334 (prec: 0.286, sens: 0.402)
Best F1 score: 0.349 (prec: 0.366, sens: 0.334)
Best F1 score: 0.389 (prec: 0.315, sens: 0.508)
Best F1 score: 0.405 (prec: 0.369, sens: 0.448)
2088
CV #2:
Best F1 score: 0.004 (prec: 0.696, sens: 0.002)
Best F1 score: 0.305 (prec: 0.303, sens: 0.308)
Best F1 score: 0.320 (prec: 0.324, sens: 0.317)
Best F1 score: 0.344 (prec: 0.345, sens: 0.344)
Best F1 score: 0.398 (prec: 0.348, sens: 0.464)
Best F1 score: 0.402 (prec: 0.337, sens: 0.496)
Best F1 score: 0.404 (prec: 0.379, sens: 0.433)
4176
CV #3:
Best F1 score: 0.018 (prec: 0.746, sens: 0.009)
Best F1 score: 0.196 (prec: 0.496, sens: 0.122)
Best F1 score: 0.353 (prec: 0.315, sens: 0.402)
Best F1 score: 0.364 (prec: 0.502, sens: 0.286)
Best F1 score: 0.365 (prec: 0.267, sens: 0.576)
Best F1 score: 0.398 (prec: 0.361, sens: 0.443)
Best F1 score: 0.406 (prec: 0.386, sens: 0.427)
Best F1 s

In [8]:
len(cv_results), df.shape

(10438, (10438, 25))

In [9]:
df_deepcoil = pd.DataFrame.from_dict(cv_results, orient='index')
df_deepcoil.columns = ['deepcoil_assignment']
df = pd.concat([df, df_deepcoil], axis=1, sort=True)

In [11]:
df.to_pickle('out/seq/cv_results.p')
df.to_csv('out/seq/cv_results.csv')

In [12]:
df_test = pickle.load(open('./../3_Train_Test_Split/out/test_data.p', 'rb'))

In [13]:
test_data = encode(df_test)

In [14]:
test_ensemble_results = {}
for i in range(1, 6):
    model.load_weights('models/final_seq/final_seq_%s.h5' % i)
    preds = model.predict(test_data[0])
    decoded_preds = decode(preds, test_data[0])
    for decoded_pred, entry in zip(decoded_preds, test_data[2]):
        if i == 1:
            test_ensemble_results[entry] = decoded_pred
        else:
            test_ensemble_results[entry] = np.vstack((test_ensemble_results[entry], decoded_pred))

In [15]:
test_avg_results = {}
for key, value in test_ensemble_results.items():
    avg_results = np.average(value, axis=0)
    assignment  = ''.join(str(int(label)) for label in np.rint(avg_results))
    test_avg_results[key] = assignment

In [16]:
len(test_avg_results), df_test.shape

(1193, (1193, 24))

In [17]:
df_deepcoil = pd.DataFrame.from_dict(test_avg_results, orient='index')
df_deepcoil.columns = ['deepcoil_assignment']
df_test = pd.concat([df_test, df_deepcoil], axis=1, sort=True)

In [18]:
df_test.to_pickle('out/seq/test_results.p')
df_test.to_csv('out/seq/test_results.csv')

#### Li (2016) benchmark dataset

In [19]:
df_test = pickle.load(open('./../3_Train_Test_Split/out/li2016.p', 'rb'))

In [20]:
test_data = encode(df_test)

In [21]:
test_ensemble_results = {}
for i in range(1, 6):
    model.load_weights('models/final_seq/final_seq_%s.h5' % i)
    preds = model.predict(test_data[0])
    decoded_preds = decode(preds, test_data[0])
    for decoded_pred, entry in zip(decoded_preds, test_data[2]):
        if i == 1:
            test_ensemble_results[entry] = decoded_pred
        else:
            test_ensemble_results[entry] = np.vstack((test_ensemble_results[entry], decoded_pred))

In [22]:
test_avg_results = {}
for key, value in test_ensemble_results.items():
    avg_results = np.average(value, axis=0)
    assignment  = ''.join(str(int(label)) for label in np.rint(avg_results))
    test_avg_results[key] = assignment

In [23]:
len(test_avg_results), df_test.shape

(518, (518, 24))

In [24]:
df_deepcoil = pd.DataFrame.from_dict(test_avg_results, orient='index')
df_deepcoil.columns = ['deepcoil_assignment']
df_test = pd.concat([df_test, df_deepcoil], axis=1, sort=True)

In [25]:
df_test.to_pickle('out/seq/li2016_results.p')
df_test.to_csv('out/seq/li2016_results.csv')